## Implement TF-IDF using Scikit-Learn Library

Documentation: [TFidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

Convert a collection of raw documents to a matrix of TF-IDF features.

In [1]:
import os
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle
import numpy as np

Before obtaining the TF-IDF matrix, make sure that text corpus is generate during preprocessing. If not, go back to [Corpus Generation](03-Corpus_Generation.ipynb).

#### 1. Calculate TF-IDF on preprocessed documents
- params: 
    - processed_doc_path: file that has been preprocessed
    - document_limit: restrict the number to process with tf-idf
- variables:
    - dictionary: set, to save unique token
    - documents_list: list, for each document, there is a documents_list to save all tokens that occured
    - files_list: to save file name

In [2]:
def scikit_tf_idf_prerocessed_doc(processed_doc_path, document_limit=200):
    
    dictionary = set()
    documents = []
    files_list = []
    
    for root, dirs, files in os.walk(processed_doc_path):
        for f in files:
            document_limit -= 1
            files.append(f)
            print('.', end='')
            words_list = ''
            with open(root+'/'+f, 'rb') as fr:
                try:
                    words = pickle.load(fr)
                    dictionary.update(words)
                    words_list = ' '.join(words)
                except:
                    print ('Error while processing file: ', f)
            
            documents.append(words_list)
            
            if document_limit == 0:
                print('\n')
                break

    sklearn_vectorizer = TfidfVectorizer(vocabulary=dictionary)
    sklearn_tf_idf = sklearn_vectorizer.fit_transform(documents)
    return sklearn_vectorizer, sklearn_tf_idf, files

In [3]:
%%time
#please modify the relative path
# processed_doc_path = '/home/bit/ma0/LabShare/data/chui_ma/spacy_corpus/'
processed_doc_path = '../spacy_corpus/'
sklearn_vectorizer, sklearn_tf_idf, files_list = scikit_tf_idf_prerocessed_doc(processed_doc_path, 200)

........................................................................................................................................................................................................

CPU times: user 2.2 s, sys: 118 ms, total: 2.32 s
Wall time: 2.25 s


#### 2. Find most important words using tf-idf
- how often does a word appear in a report => “term frequency tf ”
- how rare is a word across all reports => “inverse document frequency idf”
- => importance score = tf * idf
- params:
    - sklearn_vectorizer: a vectorizer generated by TfidfVectorizer
    - tf_idf: matrix, obtain from function: scikit_tf_idf_prerocessed_doc(processed_doc_path, document_limit=200)
    - k: number of keywords to display
    - filenames: list of file names
- variables: 
    - document_important_words: dict, key: file name, value: important words after calculating tf-idf

In [4]:
def sklearn_find_top_k_words(sklearn_vectorizer, tf_idf, k, filenames):
    
    document_important_words = {}
    corpus = sklearn_vectorizer.get_feature_names()
    
    for i in range(tf_idf.shape[1]):
        important_words = []
        
        #get indices of k-maximum values in numpy column
        index = np.argpartition(tf_idf[:, i], -k)[-k:]
        index = index[::-1]
        
        for ind in index:
            important_words.append(corpus[ind])
        
        document_important_words[filenames[i]] = important_words
        
        df = pd.DataFrame(document_important_words)
    return df

Find the most important 10 keywords in each business report, according the the term weighting matrix obtained in tf_idf

In [5]:
#find most important 10 keywords
#return pandas DataFrame
sk_tf_idf = np.transpose(sklearn_tf_idf)
sklearn_find_top_k_words(sklearn_vectorizer, sk_tf_idf.toarray(), 10, files_list)

,DeutscheWohnen-AnnualReport-2016,BayWa-QuarterlyReport-2015-Q2,Freenet-QuarterlyReport-2011-Q1,BASF-QuarterlyReport-2017-Q1,DIC-Asset-AnnualReport-2014,Fraport-QuarterlyReport-2016-Q1,Deutsche_Euroshop-QuarterlyReport-2010-Q2,Aixtron-QuarterlyReport-2011-Q1,Bechtle-QuarterlyReport-2015-Q2,Fresenius-QuarterlyReport-2017-Q1,...,BVB-QuarterlyReport-2015-Q1,Deutsche_Post-AnnualReport-2014,eon-AnnualReport-2016,ADVA_Optical-QuarterlyReport-2012-Q2,Deutsche_Post-QuarterlyReport-2014-Q1,AmadeusFiRe-QuarterlyReport-2014-Q1,Drillisch-QuarterlyReport-2013-Q1,Durr-QuarterlyReport-2013-Q3,Beiersdorf-QuarterlyReport-2014-Q1,Fresenius_Medical_Care-QuarterlyReport-2016-Q3
0,wohnungen,segment,aufgegeben,betriebstätigkeit,asset,lima,deutsch,bauelemente,prozent,care,...,borussia,dhl,abspaltung,netzbetreiber,dhl,fire,maintal,dürr,konzern,wechselkursen
1,wesentlich,baywa,vergleichsquartal,basf,frankfurt,frankfurt,euroshop,märz,tsd,quirónsalud,...,dortmund,post,prozent,optical,vorjahr,vorjahr,prozent,tsd,zwischenbericht,fresenius
2,gsw,halbjahr,prozent,chemicals,abs,passagier,wildau,herzogenrath,bechtle,medical,...,kgaa,deutsch,uniper,networking,forwarding,personalvermittlung,höhe,systems,nivea,gesundheitsdienstleistungen
3,höhe,agrar,quartal,umsatz,dic,passagiere,dresden,depositionsanlagen,vorjahr,fresenius,...,iduna,vorjahr,wesentlich,millionen,umsatz,amadeus,millionen,thermotechnik,tesa,gesellschaft
4,berlin,limited,freenet,sondereinflüssen,main,vorjahr,center,halbleiterindustrie,managed,onzern,...,signal,unternehmensbereich,unternehmen,adva,pep,zeitarbeit,freenet,bayreuth,beiersdorf,monaten
5,immobilien,energien,millionen,deutlich,immobilien,anteilsverkauf,wuppertal,gasphasenabscheidung,juni,kgaa,...,dortmunder,express,dezember,proforma,post,weiterbildung,drillisch,cleaning,vorjahr,umsatzerlöse
6,wohnen,prozent,cashﬂ,hoch,gmbh,liegen,kassel,aixtron,modus,kabi,...,spielzeit,berichtsjahr,energien,entwicklungskosten,quartal,berichtsquartal,msp,ltb,januar,niereninsuffizienz
7,vorjahr,geschäftsfeld,weitergeführt,ebit,maintor,fraport,kommanditanteile,jahresvergleich,liegen,fmch,...,september,forwarding,höhe,entwicklungsprojekte,unternehmensbereich,fakturierbaren,märz,assembly,unternehmensbereich,quartal
8,deutsch,dürfen,next,verkaufsmengen,aufsichtsrat,fcs,gesellschaft,vorquartal,quartal,helios,...,mannschaft,sar,beziehungsweise,fsp,freight,quartal,mobil,filtration,consumer,cms
9,betreut,energie,märz,vorjahresquartal,vorstand,ausgabeverhalten,bewertungsergebnis,vergleich,steigen,onzerns,...,gmbh,global,aufsichtsrat,herstellungskosten,chain,zeitarbeitsmitarbeiter,dezember,auftragseingang,liegen,konstant


After implementation of TF-IDF, we would like to further investigate on [LSI search](06-LSI_Search.ipynb), which is based on TF-IDF.